# Playground

This is a space for experimentation.

In [44]:
import random
import itertools
import numpy as np
from player import Player
from manual import Manual
from minimax import MiniMax
from strategy import Strategy
from datetime import datetime
from tic_tac_toe import WorldTTT
from utility import list_to_tuple_2d
from utility import get_opposite_symbol

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
s = list_to_tuple_2d([
    ['O', '#', '#'],
    ['#', 'X', '#'],
    ['X', '#', '#']
])

In [46]:
world_ttt = WorldTTT()

In [21]:
world_ttt.state_eval(s, 'O', True)

1.0

In [47]:
strategy_minimax = MiniMax(
    is_game_over=world_ttt.is_game_over,
    state_eval=world_ttt.state_eval,
    get_next_states=world_ttt.get_next_states,
    actions=world_ttt.actions,
    alpha_beta=True
)

In [48]:
strategy_minimax.get_move(state=s, sym='O')

[DEBUG] alpha_beta = [-inf, inf]
[DEBUG] alpha_beta = [-inf, inf]
[DEBUG] alpha_beta = [-inf, inf]
[DEBUG] alpha_beta = [-inf, -10.0]
[DEBUG] alpha_beta = [-inf, -10.0]
[DEBUG] alpha_beta = [-inf, -10.0]
[DEBUG] alpha_beta = [-inf, -10.0]
[DEBUG] alpha_beta = [-inf, -10.0]
[DEBUG] alpha_beta = [-inf, -10.0]
[DEBUG] alpha_beta = [-inf, -10.0]
[DEBUG] alpha_beta = [-inf, -10.0]
[DEBUG] alpha_beta = [-10.0, inf]
[DEBUG] alpha_beta = [-10.0, inf]
[DEBUG] alpha_beta = [-10.0, inf]
[DEBUG] alpha_beta = [-10.0, inf]
[DEBUG] alpha_beta = [-10.0, inf]
[DEBUG] alpha_beta = [-10.0, inf]
[DEBUG] alpha_beta = [0.0, inf]
[DEBUG] alpha_beta = [0.0, inf]
[DEBUG] alpha_beta = [-10.0, 0.0]
[DEBUG] alpha_beta = [-10.0, inf]
[DEBUG] alpha_beta = [-10.0, inf]
[DEBUG] alpha_beta = [-10.0, inf]
[DEBUG] alpha_beta = [-10.0, inf]
[DEBUG] alpha_beta = [0.0, inf]
[DEBUG] alpha_beta = [-10.0, 10.0]
[DEBUG] alpha_beta = [-10.0, inf]
[DEBUG] alpha_beta = [-10.0, inf]
[DEBUG] alpha_beta = [-10.0, inf]
[DEBUG] alpha_

{'f_out': 2, 'seconds': 0.02563953399658203}

In [42]:
world_ttt.actions[1]

(0, 1)